In [ ]:
import pandas as pd

# === 定義 GCS 路徑 ===
base_hosp = 'gs://ba878_mimic_data/MIMIC-IV-3.1/hosp'
base_icu  = 'gs://ba878_mimic_data/MIMIC-IV-3.1/icu'
base_ed  = 'gs://ba878_mimic_data/MIMIC-IV-ED'

# === 逐一讀取檔案 ===
print("正在讀取各資料表...")

df_patients          = pd.read_csv(f"{base_hosp}/patients_CLEAN.csv")
df_admissions        = pd.read_csv(f"{base_hosp}/admissions_CLEAN.csv")
df_transfers         = pd.read_csv(f"{base_hosp}/transfers_CLEAN.csv")
# df_d_icd_diagnoses   = pd.read_csv(f"{base_hosp}/d_icd_diagnoses.csv.gz", compression='gzip')
# df_diagnoses_icd     = pd.read_csv(f"{base_hosp}/diagnoses_icd.csv.gz", compression='gzip')
# df_d_icd_procedures  = pd.read_csv(f"{base_hosp}/d_icd_procedures.csv.gz", compression='gzip')
# df_procedures_icd    = pd.read_csv(f"{base_hosp}/procedures_icd.csv.gz", compression='gzip')
# df_drgcodes          = pd.read_csv(f"{base_hosp}/drgcodes.csv.gz", compression='gzip')
# df_poe_detail        = pd.read_csv(f"{base_hosp}/poe_detail.csv")
# df_d_hcpcs           = pd.read_csv(f"{base_hosp}/d_hcpcs.csv.gz", compression='gzip')
# df_omr               = pd.read_csv(f"{base_hosp}/omr.csv.gz", compression='gzip')

df_icustays          = pd.read_csv(f"{base_icu}/icustays_CLEAN.csv")

df_edstays           = pd.read_csv(f"{base_ed}/edstays_clean.csv", dtype={"hadm_id": "string"})
df_edstays["hadm_id"]= pd.to_numeric(df_edstays["hadm_id"].str.strip(), errors="coerce").astype("Int64")

print("✅ 全部檔案讀取完成\n")

正在讀取各資料表...
✅ 全部檔案讀取完成



In [ ]:
import pandas as pd
import numpy as np

# ========================================
# 1) 時間轉換工具函數
# ========================================
def ensure_datetime(df, cols):
    """將指定欄位轉換為 datetime 格式"""
    for c in cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

# ========================================
# 2) 資料型別清理
# ========================================

# ED stays 資料清理
df_edstays = ensure_datetime(df_edstays.copy(), ["intime", "outtime"])
df_edstays = df_edstays.astype({
    "gender": "string",
    "race": "string",
    "arrival_transport": "string",
    "disposition": "string"
})

# ICU stays 資料清理
df_icustays = ensure_datetime(df_icustays.copy(), ["intime", "outtime"])
df_icustays = df_icustays.astype({
    "first_careunit": "string",
    "last_careunit": "string"
})

# ========================================
# 3) ICU 再入院標籤計算（調整版）
# ========================================

# 確保時間格式正確並排序
df_icustays['intime'] = pd.to_datetime(df_icustays['intime'])
df_icustays['outtime'] = pd.to_datetime(df_icustays['outtime'])
df_icustays = df_icustays.sort_values(['subject_id', 'intime', 'outtime']).reset_index(drop=True)

# 計算與下一次 ICU 入院的時間差（往後看，而不是往前看）
df_icustays['next_intime'] = df_icustays.groupby('subject_id')['intime'].shift(-1)
df_icustays['time_to_next_hours'] = (df_icustays['next_intime'] - df_icustays['outtime']).dt.total_seconds() / 3600

# 標記：如果這次出院後 72 小時內有下一次入院，則這次標記為 readmission=1
# 同住院內 ICU-to-ICU 轉床：這種情況通常發生在病人於同一次住院期間在不同 ICU 之間轉科，例如從醫療 ICU 轉至外科 ICU。兩次紀錄的時間間隔通常只有幾分鐘到一小時，應被視為同一次連續的 ICU 住院，而不應算作再入院。
# 系統紀錄斷開：這是技術性問題造成的分段，例如系統自動關閉再開一筆新紀錄或跨午夜時間點導致資料被拆開。間隔多在 0 到 0.5 小時之間，也不應被視為再入院。
# 真實出 ICU 再回 ICU（轉病房後惡化）：這是臨床上真正有意義的再入院事件，病人離開 ICU 轉到普通病房後，在數小時或十幾小時內因病情惡化而再度進入 ICU。此類情況的時間間隔通常大於 3 小時，應該被納入再入院的定義。
df_icustays['readmission'] = np.where(
    (df_icustays['time_to_next_hours'] >= 1) & (df_icustays['time_to_next_hours'] < 72), 1, 0
    )

# ========================================
# 4) ICU 資料聚合
# ========================================

df_icustays_sorted = df_icustays.sort_values(["subject_id", "intime"])
icu_agg = (
    df_icustays_sorted
    .groupby(["subject_id", "hadm_id"], dropna=False, sort=False)
    .agg(
        n_icu_stays=("stay_id", "nunique"),
        first_icu_in=("intime", "min"),
        last_icu_out=("outtime", "max"),
        first_careunit=("first_careunit", "first"),
        last_careunit=("last_careunit", "last"),
        readmission=("readmission", "max")  # 改用 max，只要有一次導致 readmission 就標記
    )
    .reset_index()
)

# ========================================
# 5) ED 資料排序
# ========================================

df_edstays_sorted = df_edstays.sort_values(["subject_id", "intime"])

# ========================================
# 6) Transfer 路徑聚合
# ========================================

df_transfers = df_transfers.sort_values(["subject_id", "hadm_id", "intime"])

def _path_fix(g):
    """建立照護單位轉移路徑"""
    care = g["careunit"].tolist()
    evt = g["eventtype"].tolist()
    if care and care[-1] == "UNKNOWN" and str(evt[-1]).lower() == "discharge":
        care[-1] = "discharge"
    return "→".join(care)

transfers_agg = (
    df_transfers
    .groupby(["subject_id", "hadm_id"], dropna=False)
    .apply(_path_fix)
    .reset_index(name="careunit_path")
)

# ========================================
# 7) 合併 ED、ICU、Transfers 資料
# ========================================

ed_icu_merged = df_edstays_sorted.merge(
    icu_agg,
    on=["subject_id", "hadm_id"],
    how="left"
)

ed_icu_trans = ed_icu_merged.merge(
    transfers_agg,
    on=["subject_id", "hadm_id"],
    how="left"
)

# ========================================
# 8) 加入 admissions 和 patients 資料
# ========================================

# 選擇需要的 admission 欄位
df_admissions = df_admissions[[
    "subject_id", "hadm_id", "admittime", "dischtime", "deathtime",
    "admission_type", "admission_location", "discharge_location",
    "insurance", "language", "marital_status"
]]

# 移除重複的 gender 欄位（因為 edstays 已有）
df_patients = df_patients.drop(columns=["gender"])

# 合併所有資料
df_merged = (
    ed_icu_trans
    .merge(df_patients, on="subject_id", how="left")
    .merge(df_admissions, on=["subject_id", "hadm_id"], how="left")
)

df_merged["readmission"] = df_merged["readmission"].fillna(0).astype("int8")

# 轉換時間欄位
for col in ["admittime", "dischtime", "deathtime", "dod"]:
    if col in df_merged.columns:
        df_merged[col] = pd.to_datetime(df_merged[col], errors="coerce")

# 轉換字串欄位
df_merged = df_merged.astype({
    "admission_type": "string",
    "admission_location": "string",
    "discharge_location": "string",
    "insurance": "string",
    "language": "string",
    "anchor_year_group": "string",
    "careunit_path": "string"
})

# ========================================
# 9) 建立事件標記欄位
# ========================================

# 進 ED 後 12 小時內是否轉上 ICU
df_merged["icu_within_12h"] = (
    ((df_merged["first_icu_in"] - df_merged["intime"]) >= pd.Timedelta(0))
    & ((df_merged["first_icu_in"] - df_merged["intime"]) < pd.Timedelta(hours=12))
).astype("int8")

# 進 ED 後 30 天內是否死亡
df_merged["death_within_30d"] = (
    ((df_merged["dod"] - df_merged["intime"]) >= pd.Timedelta(0))
    & ((df_merged["dod"] - df_merged["intime"]) < pd.Timedelta(days=30))
).astype("int8")

df_merged["death_within_60d"] = (
    ((df_merged["dod"] - df_merged["intime"]) >= pd.Timedelta(0))
    & ((df_merged["dod"] - df_merged["intime"]) < pd.Timedelta(days=60))
).astype("int8")

df_merged["death_within_90d"] = (
    ((df_merged["dod"] - df_merged["intime"]) >= pd.Timedelta(0))
    & ((df_merged["dod"] - df_merged["intime"]) < pd.Timedelta(days=90))
).astype("int8")

# ========================================
# 10) 最終排序與輸出
# ========================================

df_merged = df_merged.sort_values(["subject_id", "admittime"])
main = df_merged.reset_index(drop=True)

/tmp/ipython-input-4221970857.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_path_fix)


In [ ]:
main_icu = main[main['n_icu_stays'] > 0].copy()
print(main_icu.shape)
print(main_icu['n_icu_stays'].describe())
output_path = "main_icu.parquet"
main_icu.to_parquet(output_path, index=False)


(31915, 34)
count    31915.000000
mean         1.104120
std          0.369745
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          7.000000
Name: n_icu_stays, dtype: float64


In [ ]:
cols = [
    "death_within_30d",
    "death_within_60d",
    "death_within_90d"
]

# 篩選 icu_within_12h == 1 的樣本後，再取平均
death_rates_icu12h = df_merged.loc[df_merged["icu_within_12h"] == 1, cols].mean().to_frame("mean_rate")

# 轉成百分比方便閱讀
death_rates_icu12h["mean_rate(%)"] = (death_rates_icu12h["mean_rate"] * 100).round(2)

death_rates_icu12h

,mean_rate,mean_rate(%)
death_within_30d,0.150552,15.06
death_within_60d,0.185519,18.55
death_within_90d,0.209033,20.90


In [ ]:
main.to_parquet("main.parquet", index=False)

In [ ]:
import pandas as pd
main = pd.read_parquet("gs://ba878_mimic_data/MIMIC-IV-3.1/main.parquet")

In [ ]:
df_sample = df_transfers.head(100)
output_path = "df_sample.csv"
df_sample.to_csv(output_path, index=False, encoding="utf-8-sig")

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from typing import List
from xgboost import XGBClassifier

In [ ]:
# # === Imports ===
# from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
# import matplotlib.pyplot as plt
# from typing import List
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, roc_auc_score
# from xgboost import XGBClassifier

# # === Config ===
# TARGET = 'icu_within_12h'  # 目標欄位：0/1

# # 若讀取 Parquet 需指定 engine，可在 read_parquet(..., engine='pyarrow')
# df = main

# # === 1) 去除識別欄與目標欄 ===
# drop_like = {'subject_id', 'hadm_id', 'stay_id', 'row_id', 'encounter',
#              'patientunitstayid', 'intime', 'outtime'}
# drop_cols = [c for c in df.columns if c.lower() in drop_like or c == TARGET]
# X = df.drop(columns=drop_cols, errors='ignore')
# y = df[TARGET].astype(int).values

# # === 2) 特徵前處理 ===
# # 2.1 將 datetime 轉為 epoch 秒
# for c in X.columns:
#     if pd.api.types.is_datetime64_any_dtype(X[c]):
#         X[c] = X[c].astype("int64") // 10**9

# # 2.2 將 object/boolean -> category；其餘嘗試轉數值
# cat_cols: List[str] = []
# num_cols: List[str] = []

# for c in X.columns:
#     if X[c].dtype == 'object' or str(X[c].dtype) == 'boolean':
#         X[c] = X[c].astype('category')
#         cat_cols.append(c)
#     elif pd.api.types.is_categorical_dtype(X[c]):
#         cat_cols.append(c)
#     elif pd.api.types.is_numeric_dtype(X[c]):
#         num_cols.append(c)
#     else:
#         # 嘗試轉數值，失敗則轉為類別
#         try:
#             X[c] = pd.to_numeric(X[c], errors='raise')
#             num_cols.append(c)
#         except Exception:
#             X[c] = X[c].astype('category')
#             cat_cols.append(c)

# print(f"Categorical columns ({len(cat_cols)}):", cat_cols[:20], '...')
# print(f"Numeric columns ({len(num_cols)}):", num_cols[:20], '...')

# # 2.3 確保 category dtype 正確
# for c in cat_cols:
#     if X[c].dtype != 'category':
#         X[c] = X[c].astype('category')

# # 2.4 僅保留指定欄位
# cols_to_keep = [
#     'gender', 'race', 'arrival_transport',
#     'anchor_age', 'anchor_year', 'anchor_year_group',
#     'insurance', 'language', 'marital_status'
# ]
# X = X[cols_to_keep]

# # === 3) 切分資料 ===
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # === 4) 計算比例並建立 XGBoost（原生處理類別） ===
# pos = (y_train == 1).sum()
# neg = (y_train == 0).sum()
# if pos == 0:
#     raise ValueError('訓練集中正類樣本數為 0，無法訓練。請檢查資料或調整切分。')
# ratio = neg / pos
# print(f'Class ratio (neg/pos) in train = {neg}/{pos} = {ratio:.3f} (-> scale_pos_weight)')

# clf = XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='auc',
#     random_state=42,
#     n_estimators=300,
#     learning_rate=0.05,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     tree_method='hist',
#     enable_categorical=True,
#     scale_pos_weight=ratio
# )

# clf.fit(X_train, y_train)
# print('Model trained.')

# # === 5) 評估：ROC-AUC 與分類報告 ===
# proba_test = clf.predict_proba(X_test)[:, 1]
# pred_test = (proba_test >= 0.5).astype(int)

# roc_auc = roc_auc_score(y_test, proba_test)
# print(f'ROC-AUC: {roc_auc:.4f}\n')
# print('Classification report (threshold=0.5):\n', classification_report(y_test, pred_test))


Class ratio (neg/pos) in train = 320275/19767 = 16.203 (-> scale_pos_weight)
